# Imports

In [3]:
import re
import inflect
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
STOPWORDS = stopwords.words('english')

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

device(type='cpu')

In [4]:
COLS_TO_DROP = ['PROJECT_ID','BACKERS_COUNT']
TEXT_FEATURES = ['NAME', 'DESC', 'KEYWORDS']
DATE_FEATURES = ['DEADLINE', 'STATE_CHANGED_AT', 'CREATED_AT', 'LAUNCHED_AT']
NOMINAL_FEATURES = ['DISABLE_COMMUNICATION', 'COUNTRY', 'CURRENCY']
TO_SCALE = ['GOAL', 'CREATE_LAUNCH_HOURS', 'CREATE_LAUNCH_HOURS_LOG', 'CREATE_DEADLINE_HOURS', 'CREATE_DEADLINE_HOURS_LOG', 'LAUNCHED_DEADLINE_HOURS', 'month', 'day', 'hour', 'minute', 'second']

# Load data

In [5]:
filepath = fr'/kaggle/input/kickstarter/01_df_development.pkl'
df_development = pd.read_pickle(filepath)
df_development

,PROJECT_ID,NAME,DESC,GOAL,KEYWORDS,DISABLE_COMMUNICATION,COUNTRY,CURRENCY,DEADLINE,STATE_CHANGED_AT,CREATED_AT,LAUNCHED_AT,BACKERS_COUNT,FINAL_STATUS,CREATE_LAUNCH_HOURS,CREATE_LAUNCH_HOURS_LOG,CREATE_DEADLINE_HOURS,CREATE_DEADLINE_HOURS_LOG,LAUNCHED_DEADLINE_HOURS,TRAIN_VAL_TEST_SPLIT
0,kkst124894863,Mall Of The Internet,"First Virtual Reality Mall, eCommerce, Live Au...",60000.0,mall-of-the-internet,False,US,USD,2015-04-22 18:21:56,2015-04-22 18:21:56,2014-12-14 18:32:03,2015-03-23 18:21:56,4,0,2375.831389,7.773524,3095.831389,8.038135,720.000000,Train
1,kkst456144988,"The Don't Tell Darlings' New Album: """"""""""""""""""""...","""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""...",800.0,the-dont-tell-darlings-new-album-sugar-for-sugar,False,US,USD,2011-09-23 04:00:00,2011-09-23 04:00:53,2011-07-26 19:20:33,2011-07-31 18:47:42,41,1,119.452500,4.791255,1400.657500,7.245411,1281.205000,Train
2,kkst598563858,OPEN SOURCE GAME Busters Nuts!,Big Buck Bunny mobile game is a open source AP...,10000.0,busters-nuts,False,US,USD,2013-06-20 03:19:57,2013-06-20 03:19:57,2012-08-24 17:51:43,2013-05-16 03:19:57,1,0,6345.470556,8.755654,7185.470556,8.879955,840.000000,Train
3,kkst1129209536,Send SueNami - DRAG Olympic,Olympics for Drag Queens? From Dress to Succes...,270.0,send-suenami-drag-olympic,False,GB,GBP,2014-07-18 22:20:18,2014-07-18 22:20:18,2014-07-10 21:25:03,2014-07-10 22:20:18,0,0,0.920833,0.652759,192.920833,5.267450,192.000000,Train
4,kkst8900047,Knowing Nile S,"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""...",5.0,knowing-nile-s,False,GB,GBP,2013-11-10 16:51:02,2013-11-10 16:51:02,2013-09-07 08:37:01,2013-10-11 15:51:02,0,0,823.233611,6.714454,1544.233611,7.342930,721.000000,Train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108124,kkst1377169294,Somewhere Out There,"Creating a smile, not just any smile, the smil...",250.0,somewhere-out-there,False,US,USD,2014-09-14 03:00:00,2014-09-14 03:00:10,2014-07-18 21:49:37,2014-08-22 16:55:17,24,1,835.094444,6.728742,1373.173056,7.225607,538.078611,Test
108125,kkst450010306,Time Tracker Android Application,Time tracker is an application that calculates...,5000.0,time-tracker-android-application,False,US,USD,2013-05-02 18:39:22,2013-05-02 18:39:25,2013-03-21 20:50:18,2013-04-02 18:39:22,0,0,285.817778,5.658847,1005.817778,6.914550,720.000000,Test
108126,kkst280861137,Living Ruins,A public sculpture and its documentary to show...,45000.0,living-ruins,False,CA,CAD,2015-04-22 01:13:36,2015-04-22 01:13:38,2015-03-10 02:17:17,2015-03-23 01:13:36,36,0,310.938611,5.742806,1030.938611,6.939194,720.000000,Test
108127,kkst1638562722,"iCulture Collection - fine poster prints, stic...",iCulture project is a comprehensive detailed c...,3000.0,iculture-collection-fine-poster-prints-sticker...,False,US,USD,2013-11-21 19:09:13,2013-11-12 00:39:08,2013-10-16 09:31:06,2013-10-22 18:09:13,0,0,152.635278,5.034581,873.635278,6.773807,721.000000,Test


In [6]:
df_development.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108129 entries, 0 to 108128
Data columns (total 20 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   PROJECT_ID                 108129 non-null  object        
 1   NAME                       108129 non-null  object        
 2   DESC                       108129 non-null  object        
 3   GOAL                       108129 non-null  float64       
 4   KEYWORDS                   108129 non-null  object        
 5   DISABLE_COMMUNICATION      108129 non-null  bool          
 6   COUNTRY                    108129 non-null  object        
 7   CURRENCY                   108129 non-null  object        
 8   DEADLINE                   108129 non-null  datetime64[ns]
 9   STATE_CHANGED_AT           108129 non-null  datetime64[ns]
 10  CREATED_AT                 108129 non-null  datetime64[ns]
 11  LAUNCHED_AT                108129 non-null  datetime

# Preprocessing

In [7]:
p = inflect.engine()

def convert_numbers_to_words(text):
    words = []
    for word in text.split():
        if word.isdigit():
            try:
                word = p.number_to_words(int(word))
            except Exception as e:
                continue
        else:
            # Use regular expression to separate numbers and non-numbers
            parts = re.findall(r'\d+|\D+', word)
            parts = [p.number_to_words(int(part)) if part.isdigit() else part for part in parts]
            word = ' '.join(parts)
        words.append(word)
    return ' '.join(words)

In [8]:
convert_numbers_to_words('27mpg')

'twenty-seven mpg'

In [9]:
def process_texts(texts):
    texts = texts.copy()
    texts = texts.str.replace('"', '')
    texts = texts.str.replace("'", '')
    texts = texts.apply(convert_numbers_to_words)
    texts = texts.apply(lambda text: ' '.join([word for word in text.split() if word.lower() not in STOPWORDS]))
    return texts


def get_sequences(texts, max_seq_length=None):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(texts)
    print(f'Vocab length: {len(tokenizer.word_index) + 1}')
    
    sequences = tokenizer.texts_to_sequences(texts)
    
    if max_seq_length is None:
        max_seq_length = np.max(list(map(len, sequences)))
    sequences = pad_sequences(sequences, maxlen=max_seq_length, padding='post')
    
    return sequences, max_seq_length


def onehot_encode(df, column):
    df = df.copy()
    one_hots = pd.get_dummies(df[column], prefix=column, drop_first=True)
    df = pd.concat([df, one_hots], axis=1)
    return df


def encode_dates(df, column):
    df = df.copy()
    col_dt = pd.to_datetime(df[column])
    df['month'] = col_dt.dt.month
    df['day'] = col_dt.dt.day
    df['hour'] = col_dt.dt.hour
    df['minute'] = col_dt.dt.minute
    df['second'] = col_dt.dt.second
    return df


def preprocess_inputs(df):
    df = df.copy()
    
    df = df.drop(COLS_TO_DROP, axis=1)
    
    df['KEYWORDS'] = df['KEYWORDS'].str.replace('-', ' ').str.strip()
    
    for text_feature in TEXT_FEATURES:
        df[text_feature] = process_texts(df[text_feature])
    
    for date_feature in DATE_FEATURES:
        df = encode_dates(df, date_feature)
        df = df.drop(date_feature, axis=1)
        
    for nominal_feature in NOMINAL_FEATURES:
        df = onehot_encode(df, nominal_feature)
        df = df.drop(nominal_feature, axis=1)
    
    df_train = df.query('TRAIN_VAL_TEST_SPLIT == "Train"')
    df_val = df.query('TRAIN_VAL_TEST_SPLIT == "Validation"')
    df_test = df.query('TRAIN_VAL_TEST_SPLIT == "Test"')
    
    y_train = df_train['FINAL_STATUS']
    y_val = df_val['FINAL_STATUS']
    y_test = df_test['FINAL_STATUS']
    
    X_train = df_train.drop('FINAL_STATUS', axis=1)
    X_val = df_val.drop('FINAL_STATUS', axis=1)
    X_test = df_test.drop('FINAL_STATUS', axis=1)
    
    print('Names')
    names_train, max_seq_length = get_sequences(X_train['NAME'])
    names_val, _ = get_sequences(X_val['NAME'], max_seq_length)
    names_test, _ = get_sequences(X_test['NAME'], max_seq_length)
    
    print('Descriptions')
    desc_train, max_seq_length = get_sequences(X_train['DESC'])
    desc_val, _ = get_sequences(X_val['DESC'], max_seq_length)
    desc_test, _ = get_sequences(X_test['DESC'], max_seq_length)
    
    print('Keywords')
    keywords_train, max_seq_length = get_sequences(X_train['KEYWORDS'])
    keywords_val, _ = get_sequences(X_val['KEYWORDS'], max_seq_length)
    keywords_test, _ = get_sequences(X_test['KEYWORDS'], max_seq_length)
    
    X_train = X_train.drop(TEXT_FEATURES + ['TRAIN_VAL_TEST_SPLIT'], axis=1)
    X_val = X_val.drop(TEXT_FEATURES + ['TRAIN_VAL_TEST_SPLIT'], axis=1)
    X_test = X_test.drop(TEXT_FEATURES + ['TRAIN_VAL_TEST_SPLIT'], axis=1)
    
    scaler = StandardScaler()
    scaler.fit(X_train[TO_SCALE])
    X_train[TO_SCALE] = pd.DataFrame(scaler.transform(X_train[TO_SCALE]), index=X_train.index, columns=TO_SCALE)
    X_val[TO_SCALE] = pd.DataFrame(scaler.transform(X_val[TO_SCALE]), index=X_val.index, columns=TO_SCALE)
    X_test[TO_SCALE] = pd.DataFrame(scaler.transform(X_test[TO_SCALE]), index=X_test.index, columns=TO_SCALE)
    
    return names_train, names_val, names_test,\
            desc_train, desc_val, desc_test,\
            keywords_train, keywords_val, keywords_test,\
            X_train, X_val, X_test,\
            y_train, y_val, y_test

In [17]:
USE_SAVED_INPUTS=True
if not USE_SAVED_INPUTS:
    names_train, names_val, names_test,\
            desc_train, desc_val, desc_test,\
            keywords_train, keywords_val, keywords_test,\
            X_train, X_val, X_test,\
            y_train, y_val, y_test = preprocess_inputs(df_development)
    
    names_train = torch.tensor(names_train, dtype=torch.int32)
    names_val = torch.tensor(names_val, dtype=torch.int32)
    names_test = torch.tensor(names_test, dtype=torch.int32)
    desc_train = torch.tensor(desc_train, dtype=torch.int32)
    desc_val = torch.tensor(desc_val, dtype=torch.int32)
    desc_test = torch.tensor(desc_test, dtype=torch.int32)
    keywords_train = torch.tensor(keywords_train, dtype=torch.int32)
    keywords_val = torch.tensor(keywords_val, dtype=torch.int32)
    keywords_test = torch.tensor(keywords_test, dtype=torch.int32)
    X_train = torch.tensor(X_train.to_numpy(), dtype=torch.float32)
    X_val = torch.tensor(X_val.to_numpy(), dtype=torch.float32)
    X_test = torch.tensor(X_test.to_numpy(), dtype=torch.float32)
    y_train = torch.tensor(y_train.to_numpy(), dtype=torch.float32)
    y_val = torch.tensor(y_val.to_numpy(), dtype=torch.float32)
    y_test = torch.tensor(y_test.to_numpy(), dtype=torch.float32)
    
    torch.save(names_train, '/kaggle/working/names_train.pt')
    torch.save(names_val, '/kaggle/working/names_val.pt')
    torch.save(names_test, '/kaggle/working/names_test.pt')
    torch.save(desc_train, '/kaggle/working/desc_train.pt')
    torch.save(desc_val, '/kaggle/working/desc_val.pt')
    torch.save(desc_test, '/kaggle/working/desc_test.pt')
    torch.save(keywords_train, '/kaggle/working/keywords_train.pt')
    torch.save(keywords_val, '/kaggle/working/keywords_val.pt')
    torch.save(keywords_test, '/kaggle/working/keywords_test.pt')
    torch.save(X_train, '/kaggle/working/X_train.pt')
    torch.save(X_val, '/kaggle/working/X_val.pt')
    torch.save(X_test, '/kaggle/working/X_test.pt')
    torch.save(y_train, '/kaggle/working/y_train.pt')
    torch.save(y_val, '/kaggle/working/y_val.pt')
    torch.save(y_test, '/kaggle/working/y_test.pt')
else:
    names_train = torch.load('/kaggle/input/kickstarter/names_train.pt')
    names_val = torch.load('/kaggle/input/kickstarter/names_val.pt')
    names_test = torch.load('/kaggle/input/kickstarter/names_test.pt')
    desc_train = torch.load('/kaggle/input/kickstarter/desc_train.pt')
    desc_val = torch.load('/kaggle/input/kickstarter/desc_val.pt')
    desc_test = torch.load('/kaggle/input/kickstarter/desc_test.pt')
    keywords_train = torch.load('/kaggle/input/kickstarter/keywords_train.pt')
    keywords_val = torch.load('/kaggle/input/kickstarter/keywords_val.pt')
    keywords_test = torch.load('/kaggle/input/kickstarter/keywords_test.pt')
    X_train = torch.load('/kaggle/input/kickstarter/X_train.pt')
    X_val = torch.load('/kaggle/input/kickstarter/X_val.pt')
    X_test = torch.load('/kaggle/input/kickstarter/X_test.pt')
    y_train = torch.load('/kaggle/input/kickstarter/y_train.pt')
    y_val = torch.load('/kaggle/input/kickstarter/y_val.pt')
    y_test = torch.load('/kaggle/input/kickstarter/y_test.pt')

In [18]:
print(f'{names_train.shape}')
print(f'{names_val.shape}')
print(f'{names_test.shape}')
print()
print(f'{desc_train.shape}')
print(f'{desc_val.shape}')
print(f'{desc_test.shape}')
print()
print(f'{keywords_train.shape}')
print(f'{keywords_val.shape}')
print(f'{keywords_test.shape}')
print()
print(f'{X_train.shape}')
print(f'{X_val.shape}')
print(f'{X_test.shape}')
print()
print(f'{y_train.shape}')
print(f'{y_val.shape}')
print(f'{y_test.shape}')

torch.Size([86503, 22])
torch.Size([10813, 22])
torch.Size([10813, 22])

torch.Size([86503, 46])
torch.Size([10813, 46])
torch.Size([10813, 46])

torch.Size([86503, 21])
torch.Size([10813, 21])
torch.Size([10813, 21])

torch.Size([86503, 30])
torch.Size([10813, 30])
torch.Size([10813, 30])

torch.Size([86503])
torch.Size([10813])
torch.Size([10813])


# Modelling

In [19]:
class MyModel(nn.Module):
    def __init__(self, num_features, name_vocab_size, name_embed_dim, desc_vocab_size, desc_embed_dim, keyword_vocab_size, keyword_embed_dim, num_classes):
        super(MyModel, self).__init__()

        self.X_dense1 = nn.Linear(num_features, 32)
        self.X_dense2 = nn.Linear(32, 64)
        
        self.name_embedding = nn.Embedding(name_vocab_size, name_embed_dim)
        self.name_flatten = nn.Flatten()

        self.desc_embedding = nn.Embedding(desc_vocab_size, desc_embed_dim)
        self.desc_flatten = nn.Flatten()

        self.keyword_embedding = nn.Embedding(keyword_vocab_size, keyword_embed_dim)
        self.keyword_flatten = nn.Flatten()

        self.concat = nn.Linear(5760, 128)
        self.outputs = nn.Linear(128, num_classes)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, X_inputs, name_inputs, desc_inputs, keyword_inputs):
#         print(f'{X_inputs.shape=}') # batch_size=32, num_features=30

        X_dense1_out = torch.relu(self.X_dense1(X_inputs))
#         print(f'{X_dense1_out.shape=}') # batch_size=32, 32
        
        X_dense2_out = torch.relu(self.X_dense2(X_dense1_out))
#         print(f'{X_dense2_out.shape=}') # batch_size=32, 64
        
#         print(f'{name_inputs.shape=}') # batch_size=32, 22
        name_embed_out = self.name_embedding(name_inputs) # batch_size=32, max_seq_len_names=22, embed_dim_name=64
        name_flatten_out = self.name_flatten(name_embed_out) # batch_size=32, max_seq_len_names*embed_dim_name=22*64=1408

#         print(f'{desc_inputs.shape=}') # batch_size=32, 46
        desc_embed_out = self.desc_embedding(desc_inputs)
#         print(f'{desc_embed_out.shape=}') # batch_size=32, max_seq_len_desc=46, embed_dim_desc=64
        desc_flatten_out = self.desc_flatten(desc_embed_out)
#         print(f'{desc_flatten_out.shape=}') # batch_size=32, max_seq_len_desc*embed_dim_desc=46*64=2944

#         print(f'{keyword_inputs.shape=}') # batch_size=32, 21
        keyword_embed_out = self.keyword_embedding(keyword_inputs)
#         print(f'{keyword_embed_out.shape=}') # batch_size=32, max_seq_len_keywd=21, embed_dim_keywd=64
        keyword_flatten_out = self.keyword_flatten(keyword_embed_out)
#         print(f'{keyword_flatten_out.shape=}') # batch_size=32, max_seq_len_keywd*embed_dim_keywd=21*64=1344

        concat_input = torch.cat((X_dense2_out, name_flatten_out, desc_flatten_out, keyword_flatten_out), dim=1)
        concat_out = torch.relu(self.concat(concat_input))   
        outputs = self.outputs(concat_out)
        outputs = self.sigmoid(outputs)

        return outputs

In [ ]:
max_seq_length = X_train.shape[1]
name_vocab_size = 53376
name_embed_dim = 128
desc_vocab_size = 62559
desc_embed_dim = 128
keyword_vocab_size = 55293
keyword_embed_dim = 128

num_classes = 1
learning_rate = 0.001
num_epochs = 100
batch_size = 1024

train_dataset = TensorDataset(X_train, names_train, desc_train, keywords_train, y_train)
val_dataset = TensorDataset(X_val, names_val, desc_val, keywords_val, y_val)
test_dataset = TensorDataset(X_test, names_test, desc_test, keywords_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

model = MyModel(
    max_seq_length,
    name_vocab_size,
    name_embed_dim,
    desc_vocab_size,
    desc_embed_dim,
    keyword_vocab_size,
    keyword_embed_dim,
    num_classes
)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model.to(device)

for epoch in range(1, num_epochs + 1):
    model.train()
    train_loss = 0.0
    pbar = tqdm(train_loader, total=len(train_loader))
    for X_inputs, names_inputs, desc_inputs, keywords_inputs, labels in pbar:
        X_inputs = X_inputs.to(device)
        names_inputs = names_inputs.to(device)
        desc_inputs = desc_inputs.to(device)
        keywords_inputs = keywords_inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        predictions = model(X_inputs, names_inputs, desc_inputs, keywords_inputs)
        loss = criterion(predictions.squeeze(), labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        pbar.set_description(f'Training loss: {loss.item()=}')
        
    model.eval()
    val_loss = 0.0
    val_predictions = []
    val_targets = []
    with torch.no_grad():
        for X_inputs, names_inputs, desc_inputs, keywords_inputs, labels in val_loader:
            X_inputs = X_inputs.to(device)
            names_inputs = names_inputs.to(device)
            desc_inputs = desc_inputs.to(device)
            keywords_inputs = keywords_inputs.to(device)
            labels = labels.to(device)

            predictions = model(X_inputs, names_inputs, desc_inputs, keywords_inputs)
            loss = criterion(predictions.squeeze(), labels)

            val_loss += loss.item()
            val_predictions.extend(torch.argmax(predictions, dim=1).cpu().numpy())
            val_targets.extend(labels.cpu().numpy())

    train_loss /= len(train_loader)
    val_loss /= len(val_loader)
    f1 = f1_score(val_targets, val_predictions, average='weighted')

    # Print epoch statistics
    print(f"Epoch {epoch}/{num_epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | F1 Score: {f1:.4f}")

  0%|          | 0/85 [00:00<?, ?it/s]

Epoch 1/100 | Train Loss: 0.7177 | Val Loss: 0.6808 | F1 Score: 0.5535


  0%|          | 0/85 [00:00<?, ?it/s]

Epoch 2/100 | Train Loss: 0.6749 | Val Loss: 0.6687 | F1 Score: 0.5535


  0%|          | 0/85 [00:00<?, ?it/s]

Epoch 3/100 | Train Loss: 0.6642 | Val Loss: 0.6588 | F1 Score: 0.5535


  0%|          | 0/85 [00:00<?, ?it/s]